<div style="-webkit-column-count: 3; -moz-column-count: 3; column-count: 3; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0">
    <div style="display: inline-block;" >
        <img src="../resources/images/tango_controls_logo.png" width="14" height="14">
    </div>
    <div style="display: inline-block; font-size:18px;">
        <b>Jupyter: mixing CLI & GUI for beamline controls</b>
    </div>
</div>
<div style="display: inline-block; font-size:12px;">
    N. Leclercq - 31st Tango Meeting - Florence - June 2017
</div>

# 2. The Jupyter Ecosystem

## 2.1 A bit more than the Notebook... 

The notebook is the emerging part of the Jupyter architecture. See the [Jupyter documentation](http://jupyter.readthedocs.io/en/latest/architecture/visual_overview.html) for a complete definition of the involved software components. 

<img src=../resources/images/jupyter_ecosystem.png>


## 2.2 The Jupyter Notebook

The Jupyter docs says: _**notebook documents are** both **human-readable documents** containing the analysis description and the results (figures, tables, etc..) **as well as executable documents** which can be run to perform data analysis._

The Jupyter notebook is a _**language agnostic evolution**_ of ipython notebook. Did you know that Jupyter stands for **Ju**lia-**py**thon-**R**?. There are today +60 [Jupyter kernels](https://github.com/jupyter/jupyter/wiki/Jupyter-kernels).

### 2.2.1 How does it work?

See [how jupyter works](http://jupyter.readthedocs.io/en/latest/architecture/how_jupyter_ipython_work.html) for detailed info. 
<img src=../resources/images/notebook_components.png>

A notebook is organized in _**cells**_: 
* the _**human-readable**_ part of the document is rendered by _**markdown cells**_
* the _**executable content**_ is taken in charge by _**code cells**_


### 2.2.2 Markdown Cell
* **markdown**

    <img src=https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/Markdown-mark.svg/260px-Markdown-mark.svg.png width="60">
               this text is actually rendered by a markdown cell! Just **double click** on it to edit.
    
* **html**
    <div style="font-size:16px; text-align:center">
        some <b>centered</b> text with specified font <i>size</i> and <u>style</u>
    </div>
    <div style="font-size:16px; text-align:center">
        more examples in next slides
    </div>
     
    
* **latex**
$$\frac{u^{n+1}_i - u^n_i}{\Delta t} = -\frac{1}{2} \left( \left.\frac{\partial E}{\partial x}\right|^n_i + \left.\frac{\partial E}{\partial x}\right|^n_i + \frac{\partial}{\partial x} \left[ A(u^{n+1}_i - u^n_i)\right] \right)$$

### 2.2.3 Code Cell
* code to be executed by the underlying kernel.
* read/eval/print loop (REPL)
<img src="../resources/images/NotebookFlow.svg" width=500>

In [ ]:
import datetime

def wtii():
    print("What time is it?")
    print("It's {}".format(datetime.datetime.now().strftime('%H:%M:%S')))

wtii()

## 2.3 The ipywidgets 

**ipywidgets** are the official Jupyter UI widgets. See the [ipywidgets documentation](https://ipywidgets.readthedocs.io/en/latest/index.html) for details.

What is the list of provided widgets? 

In [ ]:
from ipywidgets import Widget
Widget.widget_types

### 2.3.1 Advanced example: Grand canyon terrain modeling.

Thanks to [**Sylvain Corlay**](https://github.com/SylvainCorlay) for the following demo.
Makes use of [gdal](http://www.gdal.org) - a geospatial gata abstraction library.

In [1]:
from pythreejs import *
import numpy as np
import gdal as gd
from __future__ import print_function
from ipywidgets import Controller, FloatText, HTML, VBox
from pythreejs.install import install
from traitlets.traitlets import link, dlink

gc_ds = gd.Open('../resources/images/gc_dem.tif')
dem = gc_ds.ReadAsArray()[::20, ::20]
gt = gc_ds.GetGeoTransform()

z = (dem - np.mean(dem)) / 1000
nx, ny = z.shape

surf_g = SurfaceGeometry(z=list(z.flat), height_segments=nx - 1, width_segments=ny - 1)
surf = Mesh(geometry=surf_g, material=LambertMaterial(map=height_texture(z, colormap='terrain')), scale=(10, 10, 1))
scene = Scene(children=[AmbientLight(color='#777777'),
                        surf, 
                        DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

camera = PerspectiveCamera(position=[0, 10, 10], up=[0, 0, 1], 
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)],
                      aspect=2)
width = 950
height = 950 / camera.aspect
camera.look_at(camera.position, (1, 0, 0))
fly_controls = FlyControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, width=str(width), height=str(height), controls=[fly_controls])

In [2]:
camera.position = [15, 25, 20]
camera.look_at(camera.position, (1, 1, 1))

In [3]:
renderer

#### Could ipywidgets help us to adjust the camera position? 
#### Solution #1

In [4]:
from ipywidgets import FloatSlider, HBox, VBox

x_slider, y_slider, z_slider = (FloatSlider(description='x', min=10.0, max=20.0, orientation='horizontal'),
                                FloatSlider(description='y', min=10.0, max=20.0, orientation='horizontal'),
                                FloatSlider(description='z', min=10.0, max=20.0, orientation='horizontal'))

def update(change):
    camera.position = [x_slider.value, y_slider.value, z_slider.value]
    
x_slider.observe(update, names=['value'])
y_slider.observe(update, names=['value'])
z_slider.observe(update, names=['value'])

VBox([HBox([x_slider, y_slider, z_slider]), renderer])

#### Solution #2

In [5]:
pad = Controller()
pad

In [6]:
factor = 10
def affine(constant, factor):
    return lambda x: constant + factor * x

pad.links = []

def setup():
    if pad.connected:
        pad.links.append(dlink((pad.axes[1], 'value'), (fly_controls, 'pitch'), affine(0.0, factor)))
        pad.links.append(dlink((pad.axes[0], 'value'), (fly_controls, 'roll'), affine(0.0, -factor)))
        pad.links.append(dlink((pad.axes[3], 'value'), (fly_controls, 'forward_speed'), affine(0.0, 1 * factor)))
        pad.links.append(dlink((pad.axes[2], 'value'), (fly_controls, 'yaw'), affine(0.0, factor)))
        pad.links.append(dlink((pad.buttons[5], 'value'), (surf, 'scale'), lambda x: (10, 10, 1 - x)))
        print("gamepad up & ready!")
    if not pad.connected:
        for l in pad.links:
            l.unlink()
        pad.links = []
        print("gamepad off!")

pad.observe(setup, names=['connected'])

setup()

gamepad up & ready!


In [19]:
import ipywidgets as widgets
list = ['P0', 'P1', 'P2', 'P3', 'P4']
children = [widgets.Text(description=name) for name in list]
tab = widgets.Tab(children=children)
tab